In [1]:
%matplotlib inline
from taxi_pakage import *
taxi = pd.read_csv("edited_train.csv")

In [2]:
# 날씨 데이터 생성
weather_event = ['20160110', '20160113', '20160117', '20160123', '20160205', '20160208', '20160215', '20160216',
                 '20160224', '20160225', '20160314', '20160315', '20160328', '20160329', '20160403', '20160404',
                 '20160530', '20160628']

weather_event = pd.Series(pd.to_datetime(weather_event, format = '%Y%m%d')).dt.date
weather_event = weather_event.astype('<U32')
weather_event = list(weather_event)

taxi["y-m-d"] = pd.to_datetime(taxi["pickup_datetime"]).apply(lambda x: x.strftime("%Y-%m-%d"))
taxi["extreme_weather"] = taxi["y-m-d"].apply(lambda x: 1 if x in weather_event else 0)
taxi["weather_event"] = taxi["extreme_weather"] # 날씨 (1:자연재해,  0:자연재해X)
taxi.drop(['y-m-d', 'extreme_weather'], axis=1, inplace=True)

In [3]:
taxi['sqrt_log_dist'] = taxi['dist'].apply(lambda x: np.sqrt(np.log1p(x)))
taxi['cbrt_log_dist'] = taxi['dist'].apply(lambda x: np.cbrt(np.log1p(x)))
taxi['log_duration'] = taxi['trip_duration'].apply(lambda x: np.log1p(x))

In [4]:
taxi['velo'] = taxi['dist']/taxi['trip_duration']*3600 # 시속
taxi['no_passenger'] = taxi['passenger_count'].apply(lambda x: 1 if x == 0 else 0)

In [5]:
# 아웃라이어 제거
taxi = taxi[taxi['trip_duration'] < 1500000].reset_index(drop=True)
taxi = taxi[taxi['velo']<100]

In [7]:
taxi.columns

Index(['id', 'vendor_id', 'pickup_datetime', 'dropoff_datetime',
       'passenger_count', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'store_and_fwd_flag',
       'trip_duration', 'dist', 'month', 'day', 'hour', 'weekday', 'holiday',
       'work', 'weather_event', 'sqrt_log_dist', 'cbrt_log_dist',
       'log_duration', 'velo', 'no_passenger'],
      dtype='object')

In [14]:
filtered = taxi[taxi['velo']>3]

In [8]:
# origin data model
model = sm.OLS.from_formula("log_duration ~ scale(hour) +scale(hour**2) +scale(hour**3) + scale(hour**4) +scale(hour**5) +scale(hour**6)+scale(hour**7) + scale(hour**8) + scale(hour**9)", data = taxi)
result2 = model.fit_regularized(alpha=0.01, L1_wt=1)
print(result2.params)

Intercept           6.457340
scale(hour)         0.037837
scale(hour ** 2)    0.000000
scale(hour ** 3)    0.000000
scale(hour ** 4)    0.000000
scale(hour ** 5)    0.000000
scale(hour ** 6)   -0.013010
scale(hour ** 7)   -0.008714
scale(hour ** 8)    0.000000
scale(hour ** 9)    0.000000
dtype: float64


In [9]:
# origin data model
model = sm.OLS.from_formula("log_duration ~ scale(month) +scale(month**2) +scale(month**3) + scale(month**4) +scale(month**5) +scale(month**6)+scale(month**7) + scale(month**8) + scale(month**9)", data = taxi)
result2 = model.fit_regularized(alpha=0.01, L1_wt=1)
print(result2.params)

Intercept            6.457340
scale(month)         0.026245
scale(month ** 2)    0.000971
scale(month ** 3)    0.000000
scale(month ** 4)    0.000000
scale(month ** 5)    0.000000
scale(month ** 6)    0.000000
scale(month ** 7)    0.000000
scale(month ** 8)    0.000000
scale(month ** 9)    0.000000
dtype: float64


location

In [10]:
results = pd.DataFrame(columns = ["R-square", "AIC", "BIC", "Cond.No.", "Pb(Fstatics)", "Pb(omnibus)", "Pb(jb)", "Dub-Wat","Remarks"])

# origin data model
model1 = sm.OLS.from_formula("log_duration ~ scale(sqrt_log_dist) + scale(pickup_latitude) + scale(dropoff_latitude)+ scale(pickup_longitude)+scale(dropoff_longitude)", data = taxi)
result1 = model1.fit()
storage(result1, results, 'sqrt dist +  s(location)')

model1 = sm.OLS.from_formula("log_duration ~ scale(sqrt_log_dist) + scale(dropoff_latitude)+ scale(pickup_longitude)+scale(dropoff_longitude)", data = taxi)
result1 = model1.fit()
storage(result1, results, 'sqrt dist +  w/o p_la')

model1 = sm.OLS.from_formula("log_duration ~ scale(sqrt_log_dist) + scale(pickup_latitude) + scale(pickup_longitude)+scale(dropoff_longitude)", data = taxi)
result1 = model1.fit()
storage(result1, results, 'sqrt dist +  w/o d_la')

model1 = sm.OLS.from_formula("log_duration ~ scale(sqrt_log_dist) + scale(pickup_latitude) + scale(pickup_longitude)+scale(dropoff_longitude)", data = taxi)
result1 = model1.fit()
storage(result1, results, 'sqrt dist +  w/o p_lo')

model1 = sm.OLS.from_formula("log_duration ~ scale(sqrt_log_dist) + scale(pickup_latitude) + scale(dropoff_latitude)+ scale(pickup_longitude)", data = taxi)
result1 = model1.fit()
storage(result1, results, 'sqrt dist +  w/o d_lo')

results

,R-square,AIC,BIC,Cond.No.,Pb(Fstatics),Pb(omnibus),Pb(jb),Dub-Wat,Remarks
0,0.592,2159495.690,2159568.848,3.083,0.0,0.0,0.0,1.999,sqrt dist + s(location)
1,0.592,2159671.657,2159732.622,3.020,0.0,0.0,0.0,1.999,sqrt dist + w/o p_la
2,0.590,2166669.248,2166730.213,3.036,0.0,0.0,0.0,1.999,sqrt dist + w/o d_la
3,0.590,2166669.248,2166730.213,3.036,0.0,0.0,0.0,1.999,sqrt dist + w/o p_lo
4,0.592,2161066.150,2161127.115,1.786,0.0,0.0,0.0,1.999,sqrt dist + w/o d_lo


In [16]:
model1 = sm.OLS.from_formula("log_duration ~ \
                               scale(sqrt_log_dist)*C(vendor_id)\
                             + scale(sqrt_log_dist)*C(work)\
                             + scale(weather_event)\
                             + scale(weekday)+ scale(weekday**2)\
                             + scale(hour)+ scale(hour**6)+ scale(hour**7)\
                             + scale(month)+ scale(month**2)\
                             + scale(pickup_latitude) + scale(dropoff_latitude)+ scale(pickup_longitude) \
                             + 0", data = taxi)
result = model1.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           log_duration   R-squared:                       0.631
Model:                            OLS   Adj. R-squared:                  0.631
Method:                 Least Squares   F-statistic:                 1.469e+05
Date:                Sun, 18 Mar 2018   Prob (F-statistic):               0.00
Time:                        22:06:48   Log-Likelihood:            -1.0066e+06
No. Observations:             1458465   AIC:                         2.013e+06
Df Residuals:                 1458447   BIC:                         2.013e+06
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
=============================================================================================================
                                                coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------
C(vendor_id)[1]                               6.3574      0.001   7402.667      0.000       6.356       6.359
C(vendor_id)[2]                               6.3810      0.001   7651.786      0.000       6.379       6.383
C(work)[T.1]                                  0.2027      0.001    155.930      0.000       0.200       0.205
scale(sqrt_log_dist)                          0.6395      0.001    905.154      0.000       0.638       0.641
scale(sqrt_log_dist):C(vendor_id)[T.2]       -0.0425      0.001    -52.991      0.000      -0.044      -0.041
scale(sqrt_log_dist):C(work)[T.1]             0.0130      0.001     16.157      0.000       0.011       0.015
scale(weather_event)                         -0.0061      0.000    -14.636      0.000      -0.007      -0.005
scale(sqrt_log_dist):scale(weather_event)    -0.0010      0.000     -2.476      0.013      -0.002      -0.000
scale(weekday)                                0.1346      0.002     89.575      0.000       0.132       0.138
scale(weekday ** 2)                          -0.1170      0.002    -73.638      0.000      -0.120      -0.114
scale(hour)                                   0.1691      0.001    152.933      0.000       0.167       0.171
scale(hour ** 6)                             -0.9873      0.009   -109.512      0.000      -1.005      -0.970
scale(hour ** 7)                              0.8465      0.008    100.892      0.000       0.830       0.863
scale(month)                                  0.0190      0.002      9.805      0.000       0.015       0.023
scale(month ** 2)                             0.0068      0.002      3.482      0.000       0.003       0.011
scale(pickup_latitude)                       -0.0086      0.000    -18.300      0.000      -0.009      -0.008
scale(dropoff_latitude)                      -0.0428      0.000    -92.245      0.000      -0.044      -0.042
scale(pickup_longitude)                      -0.0153      0.000    -37.460      0.000      -0.016      -0.015
==============================================================================
Omnibus:                   712362.339   Durbin-Watson:                   1.999
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         24371942.680
Skew:                           1.723   Prob(JB):                         0.00
Kurtosis:                      22.728   Cond. No.                         50.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [17]:
model1 = sm.OLS.from_formula("log_duration ~ \
                               scale(sqrt_log_dist)*C(vendor_id)\
                             + scale(sqrt_log_dist)*C(work)\
                             + scale(sqrt_log_dist)*scale(weather_event)\
                             + scale(weekday)+ scale(weekday**2)\
                             + scale(hour)+ scale(hour**6)+ scale(hour**7)\
                             + scale(month)+ scale(month**2)\
                             + scale(pickup_latitude) + scale(dropoff_latitude)+ scale(pickup_longitude) \
                             + 0", data = filtered)
result2 = model1.fit()
result2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           log_duration   R-squared:                       0.723
Model:                            OLS   Adj. R-squared:                  0.723
Method:                 Least Squares   F-statistic:                 2.205e+05
Date:                Sun, 18 Mar 2018   Prob (F-statistic):               0.00
Time:                        22:10:44   Log-Likelihood:            -6.9565e+05
No. Observations:             1433595   AIC:                         1.391e+06
Df Residuals:                 1433577   BIC:                         1.392e+06
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
=============================================================================================================
                                                coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------
C(vendor_id)[1]                               6.3709      0.001   9047.485      0.000       6.370       6.372
C(vendor_id)[2]                               6.3719      0.001   9313.683      0.000       6.371       6.373
C(work)[T.1]                                  0.2040      0.001    191.132      0.000       0.202       0.206
scale(sqrt_log_dist)                          0.6345      0.001   1076.197      0.000       0.633       0.636
scale(sqrt_log_dist):C(vendor_id)[T.2]       -0.0021      0.001     -3.160      0.002      -0.003      -0.001
scale(sqrt_log_dist):C(work)[T.1]             0.0177      0.001     26.639      0.000       0.016       0.019
scale(weather_event)                         -0.0055      0.000    -16.074      0.000      -0.006      -0.005
scale(sqrt_log_dist):scale(weather_event)    -0.0017      0.000     -5.315      0.000      -0.002      -0.001
scale(weekday)                                0.1313      0.001    106.345      0.000       0.129       0.134
scale(weekday ** 2)                          -0.1161      0.001    -88.981      0.000      -0.119      -0.114
scale(hour)                                   0.1816      0.001    199.039      0.000       0.180       0.183
scale(hour ** 6)                             -1.0362      0.007   -139.591      0.000      -1.051      -1.022
scale(hour ** 7)                              0.8838      0.007    127.985      0.000       0.870       0.897
scale(month)                                  0.0216      0.002     13.556      0.000       0.018       0.025
scale(month ** 2)                             0.0024      0.002      1.474      0.141      -0.001       0.005
scale(pickup_latitude)                       -0.0140      0.000    -37.665      0.000      -0.015      -0.013
scale(dropoff_latitude)                      -0.0369      0.000   -101.071      0.000      -0.038      -0.036
scale(pickup_longitude)                      -0.0542      0.000   -151.623      0.000      -0.055      -0.053
==============================================================================
Omnibus:                    52891.266   Durbin-Watson:                   1.999
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           114639.588
Skew:                          -0.242   Prob(JB):                         0.00
Kurtosis:                       4.298   Cond. No.                         51.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [18]:
score, result_set = cross_validater("log_duration ~ \
                               scale(sqrt_log_dist)*C(vendor_id)\
                             + scale(sqrt_log_dist)*C(work)\
                             + scale(sqrt_log_dist)*scale(weather_event)\
                             + scale(weekday)+ scale(weekday**2)\
                             + scale(hour)+ scale(hour**6)+ scale(hour**7)\
                             + scale(month)+ scale(month**2)\
                             + scale(pickup_latitude) + scale(dropoff_latitude)+ scale(pickup_longitude) \
                             + 0", taxi, 5, r_seed=3, target_log=True)

In [19]:
result_set

,R-square,AIC,BIC,Cond.No.,Pb(Fstatics),Pb(omnibus),Pb(jb),Dub-Wat,Remarks
0,0.631,1612414.563,1612630.018,50.908,0.0,0.0,0.0,1.998,log_duration ~ ...
1,0.631,1611578.819,1611794.275,50.955,0.0,0.0,0.0,2.000,log_duration ~ ...
2,0.631,1610687.359,1610902.815,50.943,0.0,0.0,0.0,1.999,log_duration ~ ...
3,0.632,1607032.039,1607247.495,50.933,0.0,0.0,0.0,1.999,log_duration ~ ...
4,0.631,1610471.057,1610686.512,50.960,0.0,0.0,0.0,2.000,log_duration ~ ...


In [20]:
score

[0.48104910830304554,
 0.4816145655639277,
 0.48229698387921804,
 0.4855740619608032,
 0.4824363062250503]

---

In [6]:
t2 = taxi.loc[:600000]

In [11]:
model1 = sm.OLS.from_formula("log_duration ~ \
                               scale(sqrt_log_dist)*C(vendor_id)\
                             + scale(sqrt_log_dist)*scale(work)\
                             + scale(sqrt_log_dist)*scale(month)\
                             + C(hour)\
                             + scale(weather_event)\
                             + scale(weekday)+ scale(weekday**2)\
                             + scale(pickup_latitude) + scale(dropoff_latitude)+ scale(pickup_longitude) \
                             + 0", data = t2)
result = model1.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           log_duration   R-squared:                       0.639
Model:                            OLS   Adj. R-squared:                  0.639
Method:                 Least Squares   F-statistic:                 2.946e+04
Date:                Sun, 18 Mar 2018   Prob (F-statistic):               0.00
Time:                        23:24:55   Log-Likelihood:            -4.0746e+05
No. Observations:              599932   AIC:                         8.150e+05
Df Residuals:                  599895   BIC:                         8.154e+05
Df Model:                          36                                         
Covariance Type:            nonrobust                                         
==========================================================================================================
                                             coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------
C(vendor_id)[1]                            6.3716      0.003   1849.544      0.000       6.365       6.378
C(vendor_id)[2]                            6.3934      0.003   1864.891      0.000       6.387       6.400
C(hour)[T.1]                              -0.0462      0.005     -9.257      0.000      -0.056      -0.036
C(hour)[T.2]                              -0.0853      0.006    -15.491      0.000      -0.096      -0.075
C(hour)[T.3]                              -0.1215      0.006    -19.919      0.000      -0.133      -0.110
C(hour)[T.4]                              -0.1701      0.007    -25.151      0.000      -0.183      -0.157
C(hour)[T.5]                              -0.2779      0.007    -40.485      0.000      -0.291      -0.264
C(hour)[T.6]                              -0.1805      0.005    -34.456      0.000      -0.191      -0.170
C(hour)[T.7]                               0.0692      0.005     15.141      0.000       0.060       0.078
C(hour)[T.8]                               0.0795      0.005     16.989      0.000       0.070       0.089
C(hour)[T.9]                               0.1287      0.005     27.832      0.000       0.120       0.138
C(hour)[T.10]                              0.1383      0.005     29.826      0.000       0.129       0.147
C(hour)[T.11]                              0.1708      0.005     37.243      0.000       0.162       0.180
C(hour)[T.12]                              0.1826      0.005     40.275      0.000       0.174       0.192
C(hour)[T.13]                              0.1705      0.005     37.540      0.000       0.162       0.179
C(hour)[T.14]                              0.1825      0.005     40.493      0.000       0.174       0.191
C(hour)[T.15]                              0.1931      0.005     42.523      0.000       0.184       0.202
C(hour)[T.16]                              0.1547      0.005     33.397      0.000       0.146       0.164
C(hour)[T.17]                              0.1532      0.004     34.107      0.000       0.144       0.162
C(hour)[T.18]                              0.1294      0.004     29.587      0.000       0.121       0.138
C(hour)[T.19]                              0.0466      0.004     10.641      0.000       0.038       0.055
C(hour)[T.20]                             -0.0387      0.004     -8.714      0.000      -0.047      -0.030
C(hour)[T.21]                              0.0856      0.004     20.616      0.000       0.077       0.094
C(hour)[T.22]                              0.0825      0.004     19.730      0.000       0.074       0.091
C(hour)[T.23]                              0.0459      0.004     10.686      0.000       0.038       0.054
scale(sqrt_log_dist)                       0.6539      0.001    699.156      0.000     

In [8]:
test = pd.read_csv("edited_test.csv")
test['sqrt_log_dist'] = test['dist'].apply(lambda x: np.sqrt(np.log1p(x)))

In [25]:
# 테스트 데이터를 통해 y값 예측
y_hat = result.predict(test)
y_hat = y_hat.apply(lambda x: (round(np.exp(x))))
ans = pd.concat([test['id'], y_hat], axis=1)
ans.rename(columns={'id':'id' , 0:'trip_duration'}, inplace=True)
ans.tail()

,id,trip_duration
625129,id3008929,311.0
625130,id3700764,1091.0
625131,id2568735,1324.0
625132,id1384355,2057.0
625133,id0621643,1040.0


In [31]:
test.loc[324125]

id                              id0505885
vendor_id                               2
pickup_datetime       2016-03-29 14:28:46
passenger_count                         2
pickup_longitude                 -121.933
pickup_latitude                   37.3896
dropoff_longitude                -121.933
dropoff_latitude                  37.3894
store_and_fwd_flag                      0
dist                            0.0295004
month                                   3
day                                    29
hour                                   14
weekday                                 1
holiday                                 0
work                                    1
weather_event                           1
sqrt_log_dist                     0.17051
Name: 324125, dtype: object

In [29]:
ans[ans['trip_duration']>50000]

,id,trip_duration
32178,id1627730,2.603122e+06
324125,id0505885,7.245892e+20


In [33]:
ans['trip_duration'].loc[324125] = 0

/home/mk/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [34]:
ans['trip_duration'].loc[324125]

0.0

In [35]:
ans['trip_duration'] = ans['trip_duration'].apply(lambda x: int(x))

In [37]:
# Kaggle 제출파일
ans.to_csv('basic_model.csv', index=False)

0.48361 - 758/1257 (60%)

In [38]:
758/1257

0.6030230708035004